In [1]:
import h2o
import time
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [2]:
# Explore a typical Data Science workflow with H2O and Python
#
# Goal: assist the manager of CitiBike of NYC to load-balance the bicycles
# across the CitiBike network of stations, by predicting the number of bike
# trips taken from the station every day.  Use 10 million rows of historical
# data, and eventually add weather data.


# Connect to a cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 mins 11 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,3 minutes
H2O cluster name:,pasha
H2O cluster total nodes:,1
H2O cluster free memory:,3.399 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [3]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.

# Set this to True if you want to fetch the data directly from S3.
# This is useful if your cluster is running in EC2.
data_source_is_s3 = False

def mylocate(s):
    if data_source_is_s3:
        return "s3n://h2o-public-test-data/" + s
    else:
        return _locate(s)

In [4]:
# Pick either the big or the small demo.
# Big data is 10M rows
small_test = [mylocate("bigdata/laptop/citibike-nyc/2013-10.csv")]
big_test =   [mylocate("bigdata/laptop/citibike-nyc/2013-07.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-08.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-09.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-10.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-11.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-12.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-01.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-02.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-03.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-04.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-05.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-06.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-07.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-08.csv")]

# ----------

# 1- Load data - 1 row per bicycle trip.  Has columns showing the start and end
# station, trip duration and trip start time and day.  The larger dataset
# totals about 10 million rows
print("Import and Parse bike data")
data = h2o.import_file(path=big_test)

Import and Parse bike data
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
# ----------

# 2- light data munging: group the bike starts per-day, converting the 10M rows
# of trips to about 140,000 station&day combos - predicting the number of trip
# starts per-station-per-day.

# Convert start time to: Day since the Epoch
startime = data["starttime"]
secsPerDay = 1000 * 3600 * 24
data["Days"] = (startime.asnumeric() / secsPerDay).floor()
data.describe()

Rows:10407546
Cols:16




,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Days
type,int,time,time,int,enum,real,real,int,enum,real,real,int,enum,int,int,int
mins,60.0,1372636800000.0,1372637042000.0,72.0,,40.680342423,-74.01713445,72.0,,40.680342423,-74.01713445,14529.0,,1899.0,0.0,15887.0
mean,868.9687260570365,1390974078989.41,1390974947925.281,444.85883540654095,,40.734381982315185,-73.99105701820217,445.2597855440662,,40.73408688953741,-73.99117077985979,17895.66183584484,,1975.7989239404546,1.0841465413652758,16098.629260922817
maxs,6250750.0,1409529587000.0,1409538405000.0,3002.0,,40.771522000000004,-73.9500479759,3002.0,,40.771522000000004,-73.9500479759,21689.0,,1998.0,2.0,16313.0
sigma,2985.105405320145,11806736501.937712,11806714056.539324,355.7559897645294,,0.01971005087361252,0.012345332018503775,360.0703808439832,,0.019730957863268683,0.012431186159808448,1938.8051788415307,,11.132784904986751,0.5630197777940005,136.6534484381553
zeros,0,0,0,0,,0,0,0,,0,0,0,,0,1248517,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,1247644,0,0
0,634.0,2013-07-01 00:00:00,2013-07-01 00:10:34,164.0,E 47 St & 2 Ave,40.75323098,-73.97032517,504.0,1 Ave & E 15 St,40.732218530000004,-73.98165557,16950.0,Customer,nan,0.0,15887.0
1,1547.0,2013-07-01 00:00:02,2013-07-01 00:25:49,388.0,W 26 St & 10 Ave,40.749717753000006,-74.002950346,459.0,W 20 St & 11 Ave,40.746745,-74.007756,19816.0,Customer,nan,0.0,15887.0
2,178.0,2013-07-01 00:01:04,2013-07-01 00:04:02,293.0,Lafayette St & E 8 St,40.73028666,-73.9907647,237.0,E 11 St & 2 Ave,40.730473090000004,-73.98672378,14548.0,Subscriber,1980.0,2.0,15887.0


In [6]:
# Now do a monster Group-By.  Count bike starts per-station per-day.  Ends up
# with about 340 stations times 400 days (140,000 rows).  This is what we want
# to predict.
grouped = data.group_by(["Days","start station name"])
bpd = grouped.count().get_frame() # Compute bikes-per-day
bpd.set_name(2,"bikes")
bpd.show()
bpd.describe()
bpd.dim

Days,start station name,bikes
15887,1 Ave & E 15 St,74
15887,1 Ave & E 18 St,51
15887,1 Ave & E 30 St,66
15887,1 Ave & E 44 St,56
15887,10 Ave & W 28 St,51
15887,11 Ave & W 27 St,65
15887,11 Ave & W 41 St,53
15887,12 Ave & W 40 St,36
15887,2 Ave & E 31 St,96
15887,2 Ave & E 58 St,103


Rows:138795
Cols:3




,Days,start station name,bikes
type,int,enum,int
mins,15887.0,,1.0
mean,16099.488166000216,,74.98502107424619
maxs,16313.0,,668.0
sigma,123.39632568805678,,64.73186265524505
zeros,0,,0
missing,0,0,0
0,15887.0,1 Ave & E 15 St,74.0
1,15887.0,1 Ave & E 18 St,51.0
2,15887.0,1 Ave & E 30 St,66.0


[138795, 3]

In [7]:
# Quantiles: the data is fairly unbalanced; some station/day combos are wildly
# more popular than others.
print("Quantiles of bikes-per-day")
bpd["bikes"].quantile().show()

Quantiles of bikes-per-day


Probs,bikesQuantiles
0.01,2
0.1,11
0.25,26
0.333,35
0.5,59
0.667,89
0.75,107
0.9,158
0.99,293


In [8]:
# A little feature engineering
# Add in month-of-year (seasonality; fewer bike rides in winter than summer)
secs = bpd["Days"]*secsPerDay
bpd["Month"]     = secs.month().asfactor()
# Add in day-of-week (work-week; more bike rides on Sunday than Monday)
bpd["DayOfWeek"] = secs.dayOfWeek()
print("Bikes-Per-Day")
bpd.describe()

Bikes-Per-Day
Rows:138795
Cols:5




,Days,start station name,bikes,Month,DayOfWeek
type,int,enum,int,enum,enum
mins,15887.0,,1.0,,
mean,16099.488166000216,,74.98502107424619,,
maxs,16313.0,,668.0,,
sigma,123.39632568805678,,64.73186265524505,,
zeros,0,,0,,
missing,0,0,0,0,0
0,15887.0,1 Ave & E 15 St,74.0,7,Mon
1,15887.0,1 Ave & E 18 St,51.0,7,Mon
2,15887.0,1 Ave & E 30 St,66.0,7,Mon


In [9]:
# ----------
# 3- Fit a model on train; using test as validation

# Function for doing class test/train/holdout split
def split_fit_predict(data):
  global gbm0,drf0,glm0,dl0
  # Classic Test/Train split
  r = data['Days'].runif()   # Random UNIForm numbers, one per row
  train = data[  r  < 0.6]
  test  = data[(0.6 <= r) & (r < 0.9)]
  hold  = data[ 0.9 <= r ]
  print("Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow)
  bike_names_x = data.names
  bike_names_x.remove("bikes")
  
  # Run GBM
  s = time.time()
  
  gbm0 = H2OGradientBoostingEstimator(ntrees=500, # 500 works well
                                      max_depth=6,
                                      learn_rate=0.1)
    

  gbm0.train(x               =bike_names_x,
             y               ="bikes",
             training_frame  =train,
             validation_frame=test)

  gbm_elapsed = time.time() - s

  # Run DRF
  s = time.time()
    
  drf0 = H2ORandomForestEstimator(ntrees=250, max_depth=30)

  drf0.train(x               =bike_names_x,
             y               ="bikes",
             training_frame  =train,
             validation_frame=test)
    
  drf_elapsed = time.time() - s 
    
    
  # Run GLM
  if "WC1" in bike_names_x: bike_names_x.remove("WC1")
  s = time.time()

  glm0 = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")
    
  glm0.train(x               =bike_names_x,
             y               ="bikes",
             training_frame  =train,
             validation_frame=test)

  glm_elapsed = time.time() - s
  
  # Run DL
  s = time.time()

  dl0 = H2ODeepLearningEstimator(hidden=[50,50,50,50], epochs=50)
    
  dl0.train(x               =bike_names_x,
            y               ="bikes",
            training_frame  =train,
            validation_frame=test)
    
  dl_elapsed = time.time() - s
  
  # ----------
  # 4- Score on holdout set & report
  train_mse_gbm = gbm0.model_performance(train).mse()
  test_mse_gbm  = gbm0.model_performance(test ).mse()
  hold_mse_gbm  = gbm0.model_performance(hold ).mse()
#   print "GBM mse TRAIN=",train_mse_gbm,", mse TEST=",test_mse_gbm,", mse HOLDOUT=",hold_mse_gbm
  
  train_mse_drf = drf0.model_performance(train).mse()
  test_mse_drf  = drf0.model_performance(test ).mse()
  hold_mse_drf  = drf0.model_performance(hold ).mse()
#   print "DRF mse TRAIN=",train_mse_drf,", mse TEST=",test_mse_drf,", mse HOLDOUT=",hold_mse_drf
  
  train_mse_glm = glm0.model_performance(train).mse()
  test_mse_glm  = glm0.model_performance(test ).mse()
  hold_mse_glm  = glm0.model_performance(hold ).mse()
#   print "GLM mse TRAIN=",train_mse_glm,", mse TEST=",test_mse_glm,", mse HOLDOUT=",hold_mse_glm
    
  train_mse_dl = dl0.model_performance(train).mse()
  test_mse_dl  = dl0.model_performance(test ).mse()
  hold_mse_dl  = dl0.model_performance(hold ).mse()
#   print " DL mse TRAIN=",train_mse_dl,", mse TEST=",test_mse_dl,", mse HOLDOUT=",hold_mse_dl
    
  # make a pretty HTML table printout of the results

  header = ["Model", "mse TRAIN", "mse TEST", "mse HOLDOUT", "Model Training Time (s)"]
  table  = [
            ["GBM", train_mse_gbm, test_mse_gbm, hold_mse_gbm, round(gbm_elapsed,3)],
            ["DRF", train_mse_drf, test_mse_drf, hold_mse_drf, round(drf_elapsed,3)],
            ["GLM", train_mse_glm, test_mse_glm, hold_mse_glm, round(glm_elapsed,3)],
            ["DL ", train_mse_dl,  test_mse_dl,  hold_mse_dl , round(dl_elapsed,3) ],
           ]
  h2o.display.H2ODisplay(table,header)
  # --------------

In [10]:
# Split the data (into test & train), fit some models and predict on the holdout data
split_fit_predict(bpd)
# Here we see an r^2 of 0.91 for GBM, and 0.71 for GLM.  This means given just
# the station, the month, and the day-of-week we can predict 90% of the
# variance of the bike-trip-starts.

Training data has 5 columns and 83197 rows, test has 41792 rows, holdout has 13806
gbm Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%


Model,mse TRAIN,mse TEST,mse HOLDOUT,Model Training Time (s)
GBM,144.1978206,337.7233790,324.8676731,12.454
DRF,599.8996508,723.9889710,685.0177303,19.764
GLM,967.1714929,957.3634230,938.1439730,0.454
DL,529.4687702,622.4935714,607.3274025,59.935


In [11]:
# ----------
# 5- Now lets add some weather
# Load weather data
wthr1 = h2o.import_file(path=[mylocate("bigdata/laptop/citibike-nyc/31081_New_York_City__Hourly_2013.csv"),
                               mylocate("bigdata/laptop/citibike-nyc/31081_New_York_City__Hourly_2014.csv")])
# Peek at the data
wthr1.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:17520
Cols:50




,Year Local,Month Local,Day Local,Hour Local,Year UTC,Month UTC,Day UTC,Hour UTC,Cavok Reported,Cloud Ceiling (m),Cloud Cover Fraction,Cloud Cover Fraction 1,Cloud Cover Fraction 2,Cloud Cover Fraction 3,Cloud Cover Fraction 4,Cloud Cover Fraction 5,Cloud Cover Fraction 6,Cloud Height (m) 1,Cloud Height (m) 2,Cloud Height (m) 3,Cloud Height (m) 4,Cloud Height (m) 5,Cloud Height (m) 6,Dew Point (C),Humidity Fraction,Precipitation One Hour (mm),Pressure Altimeter (mbar),Pressure Sea Level (mbar),Pressure Station (mbar),Snow Depth (cm),Temperature (C),Visibility (km),Weather Code 1,Weather Code 1/ Description,Weather Code 2,Weather Code 2/ Description,Weather Code 3,Weather Code 3/ Description,Weather Code 4,Weather Code 4/ Description,Weather Code 5,Weather Code 5/ Description,Weather Code 6,Weather Code 6/ Description,Weather Code Most Severe / Icon Code,Weather Code Most Severe,Weather Code Most Severe / Description,Wind Direction (degrees),Wind Gust (m/s),Wind Speed (m/s)
type,int,int,int,int,int,int,int,int,int,real,real,real,real,real,int,int,int,real,real,real,int,int,int,real,real,real,real,int,int,int,real,real,int,enum,int,enum,int,enum,int,enum,int,enum,int,enum,int,int,enum,int,real,real
mins,2013.0,1.0,1.0,0.0,2013.0,1.0,1.0,0.0,0.0,61.0,0.0,0.0,0.25,0.5,NaN,NaN,NaN,60.96,213.36,365.76,NaN,NaN,NaN,-26.700000000000003,0.12510000000000002,0.0,983.2949000000001,NaN,NaN,NaN,-15.600000000000001,0.001,1.0,,1.0,,1.0,,1.0,,1.0,,3.0,,0.0,1.0,,10.0,7.2,0.0
mean,2013.5,6.5260273972602745,15.72054794520548,11.500000000000004,2013.5005707762557,6.525114155251141,15.72134703196347,11.500114155251142,0.0,1306.3119584569736,0.4167424905220181,0.3612073490813649,0.8724453840732911,0.9630456852791879,0.0,0.0,0.0,1293.9822681953192,1643.7390016566796,2084.8938637563456,0.0,0.0,0.0,4.313046467655992,0.5967363891594567,1.3799301075268817,1017.8258144055944,0.0,0.0,0.0,12.578909070073914,14.391442968202009,4.84251968503937,,3.6586768935762226,,2.8466076696165192,,2.0114942528735633,,4.125,,3.0,,1.3784817351598173,4.84251968503937,,194.69525681985743,9.422169480726348,2.4103288784874057
maxs,2014.0,12.0,31.0,23.0,2015.0,12.0,31.0,23.0,0.0,3657.6000000000004,1.0,1.0,1.0,1.0,NaN,NaN,NaN,3657.5999,3657.5999,3657.5999,NaN,NaN,NaN,24.400000000000002,1.0,26.924,1042.2113,NaN,NaN,NaN,36.1,16.0934,60.0,,60.0,,36.0,,27.0,,27.0,,3.0,,16.0,60.0,,360.0,20.580000000000002,10.8
sigma,0.500014270017262,3.447949723847773,8.796498048523272,6.922384111875021,0.50058441171579,3.447824054577647,8.795614888684717,6.922301652025526,0.0,995.3398569657211,0.4627208309925301,0.42770569708047684,0.19715569036704708,0.08610155981044185,-0.0,-0.0,-0.0,962.7430958537232,916.7386134899587,887.2158475113932,-0.0,-0.0,-0.0,10.973128209713666,0.18579201186573496,2.5621512917896463,7.464516971789659,-0.0,-0.0,-0.0,10.039673953091574,3.6989362303340494,5.704865769828319,,6.133862539123368,,5.805532863642112,,3.1234084426128437,,6.15223536610881,,0.0,,4.073860627017756,5.704865769828319,,106.3500000314393,1.8151187111524154,1.614697905241178
zeros,0,0,0,730,0,0,0,730,17455,0,8758,8758,0,0,0,0,0,0,0,0,0,0,0,268,0,501,0,0,0,0,269,0,0,,0,,0,,0,,0,,0,,14980,0,,0,0,2768
missing,0,0,0,0,0,0,0,0,65,10780,375,375,14682,16535,17520,17520,17520,9103,14683,16535,17520,17520,17520,67,67,15660,360,17520,17520,17520,67,412,14980,14980,16477,16477,17181,17181,17433,17433,17504,17504,17518,17518,0,14980,14980,9382,14381,1283
0,2013.0,1.0,1.0,0.0,2013.0,1.0,1.0,5.0,0.0,2895.6000000000004,1.0,0.9,1.0,nan,nan,nan,nan,2895.5999,3352.8,nan,nan,nan,nan,-5.0,0.5447000000000001,nan,1013.0917000000001,nan,nan,nan,3.3000000000000003,16.0934,nan,,nan,,nan,,nan,,nan,,nan,,0.0,nan,,nan,nan,2.57
1,2013.0,1.0,1.0,1.0,2013.0,1.0,1.0,6.0,0.0,3048.0,1.0,1.0,nan,nan,nan,nan,nan,3048.0,nan,nan,nan,nan,nan,-4.4,0.5463,nan,1012.0759,nan,nan,nan,3.9000000000000004,16.0934,nan,,nan,,nan,,nan,,nan,,nan,,0.0,nan,,260.0,9.77,4.63
2,2013.0,1.0,1.0,2.0,2013.0,1.0,1.0,7.0,0.0,1828.8000000000002,1.0,1.0,nan,nan,nan,nan,n

In [12]:
# Lots of columns in there!  Lets plan on converting to time-since-epoch to do
# a 'join' with the bike data, plus gather weather info that might affect
# cyclists - rain, snow, temperature.  Alas, drop the "snow" column since it's
# all NA's.  Also add in dew point and humidity just in case.  Slice out just
# the columns of interest and drop the rest.
wthr2 = wthr1[["Year Local","Month Local","Day Local","Hour Local","Dew Point (C)","Humidity Fraction","Precipitation One Hour (mm)","Temperature (C)","Weather Code 1/ Description"]]

wthr2.set_name(wthr2.names.index("Precipitation One Hour (mm)"), "Rain (mm)")
wthr2.set_name(wthr2.names.index("Weather Code 1/ Description"), "WC1")
wthr2.describe()
# Much better!  

Rows:17520
Cols:9




,Year Local,Month Local,Day Local,Hour Local,Dew Point (C),Humidity Fraction,Rain (mm),Temperature (C),WC1
type,int,int,int,int,real,real,real,real,enum
mins,2013.0,1.0,1.0,0.0,-26.700000000000003,0.12510000000000002,0.0,-15.600000000000001,
mean,2013.5,6.5260273972602745,15.72054794520548,11.500000000000004,4.313046467655992,0.5967363891594567,1.3799301075268817,12.578909070073914,
maxs,2014.0,12.0,31.0,23.0,24.400000000000002,1.0,26.924,36.1,
sigma,0.500014270017262,3.447949723847773,8.796498048523272,6.922384111875021,10.973128209713666,0.18579201186573496,2.5621512917896463,10.039673953091574,
zeros,0,0,0,730,268,0,501,269,
missing,0,0,0,0,67,67,15660,67,14980
0,2013.0,1.0,1.0,0.0,-5.0,0.5447000000000001,nan,3.3000000000000003,
1,2013.0,1.0,1.0,1.0,-4.4,0.5463,nan,3.9000000000000004,
2,2013.0,1.0,1.0,2.0,-3.3000000000000003,0.619,nan,3.3000000000000003,


In [13]:
# Filter down to the weather at Noon
wthr3 = wthr2[ wthr2["Hour Local"]==12 ]

In [14]:
# Lets now get Days since the epoch... we'll convert year/month/day into Epoch
# time, and then back to Epoch days.  Need zero-based month and days, but have
# 1-based.
wthr3["msec"] = h2o.H2OFrame.mktime(year=wthr3["Year Local"], month=wthr3["Month Local"]-1, day=wthr3["Day Local"]-1, hour=wthr3["Hour Local"])
secsPerDay=1000*60*60*24
wthr3["Days"] = (wthr3["msec"]/secsPerDay).floor()
wthr3.describe()
# msec looks sane (numbers like 1.3e12 are in the correct range for msec since
# 1970).  Epoch Days matches closely with the epoch day numbers from the
# CitiBike dataset.  

Rows:730
Cols:11




,Year Local,Month Local,Day Local,Hour Local,Dew Point (C),Humidity Fraction,Rain (mm),Temperature (C),WC1,msec,Days
type,int,int,int,int,real,real,real,real,enum,int,int
mins,2013.0,1.0,1.0,12.0,-26.700000000000003,0.1723,0.0,-13.9,,1357070400000.0,15706.0
mean,2013.5,6.526027397260274,15.72054794520548,12.0,4.230123796423659,0.539728198074278,1.5312571428571429,14.068775790921595,,1388560852602.7397,16070.5
maxs,2014.0,12.0,31.0,12.0,23.3,1.0,12.446,34.4,,1420056000000.0,16435.0
sigma,0.5003428180039172,3.450215293068149,8.802278027009615,0.0,11.106296472475226,0.17994502792324327,2.3606424861490587,10.398985514891212,,18219740080.410755,210.87713642466474
zeros,0,0,0,0,14,0,15,7,,0,0
missing,0,0,0,0,3,3,660,3,620,0,0
0,2013.0,1.0,1.0,12.0,-3.3000000000000003,0.5934,nan,3.9000000000000004,,1357070400000.0,15706.0
1,2013.0,1.0,2.0,12.0,-11.700000000000001,0.4806,nan,-2.2,,1357156800000.0,15707.0
2,2013.0,1.0,3.0,12.0,-10.600000000000001,0.5248,nan,-2.2,,1357243200000.0,15708.0


In [15]:
# Lets drop off the extra time columns to make a easy-to-handle dataset.
wthr4 = wthr3.drop("Year Local").drop("Month Local").drop("Day Local").drop("Hour Local").drop("msec")

In [16]:
# Also, most rain numbers are missing - lets assume those are zero rain days
rain = wthr4["Rain (mm)"]
rain[ rain.isna() ] = 0
wthr4["Rain (mm)"] = rain

In [17]:
# ----------
# 6 - Join the weather data-per-day to the bike-starts-per-day
print("Merge Daily Weather with Bikes-Per-Day")
bpd_with_weather = bpd.merge(wthr4,all_x=True,all_y=False)
bpd_with_weather.describe()
bpd_with_weather.show()

Merge Daily Weather with Bikes-Per-Day
Rows:138795
Cols:10




,Days,start station name,bikes,Month,DayOfWeek,Dew Point (C),Humidity Fraction,Rain (mm),Temperature (C),WC1
type,int,enum,int,enum,enum,real,real,real,real,enum
mins,15887.0,,1.0,,,-26.700000000000003,0.1723,0.0,-13.9,
mean,16099.488166000216,,74.98502107424619,,,5.451762870514826,0.5321604665675,0.08503267408768331,15.614519464499507,
maxs,16313.0,,668.0,,,23.3,1.0,8.382,34.4,
sigma,123.39632568805678,,64.73186265524505,,,11.723905010415397,0.1784104767702021,0.5764961942157182,10.928653577314824,
zeros,0,,0,,,1956,0,130793,1567,
missing,0,0,0,0,0,980,980,0,980,118772
0,15887.0,1 Ave & E 15 St,74.0,7,Mon,21.700000000000003,0.9354,4.572,22.8,rain
1,15887.0,1 Ave & E 18 St,51.0,7,Mon,21.700000000000003,0.9354,4.572,22.8,rain
2,15887.0,1 Ave & E 30 St,66.0,7,Mon,21.700000000000003,0.9354,4.572,22.8,rain


Days,start station name,bikes,Month,DayOfWeek,Dew Point (C),Humidity Fraction,Rain (mm),Temperature (C),WC1
15887,1 Ave & E 15 St,74,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,1 Ave & E 18 St,51,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,1 Ave & E 30 St,66,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,1 Ave & E 44 St,56,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,10 Ave & W 28 St,51,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,11 Ave & W 27 St,65,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,11 Ave & W 41 St,53,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,12 Ave & W 40 St,36,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,2 Ave & E 31 St,96,7,Mon,21.7,0.9354,4.572,22.8,rain
15887,2 Ave & E 58 St,103,7,Mon,21.7,0.9354,4.572,22.8,rain


In [18]:
# 7 - Test/Train split again, model build again, this time with weather
split_fit_predict(bpd_with_weather)

Training data has 10 columns and 83276 rows, test has 41631 rows, holdout has 13888
gbm Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%


Model,mse TRAIN,mse TEST,mse HOLDOUT,Model Training Time (s)
GBM,120.3078530,291.8993667,285.0585033,19.141
DRF,117.6329195,388.7562018,385.6955657,88.24
GLM,862.8161359,879.2555091,875.8816961,0.449
DL,238.1676790,340.7971229,351.0403636,63.611
